# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75.

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели.
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [ ]:
!pip install pandas
!pip install warnings
!pip install numpy
!pip install optuna
!pip install re
!pip install collections
!pip install torch
!pip install math
!pip install matplotlib
!pip install sklearn
!pip install spacy
!pip install tqdm
!pip install nltk
!pip install imblearn

ERROR: Could not find a version that satisfies the requirement warnings (from versions: none)
ERROR: No matching distribution found for warnings
     |████████████████████████████████| 390 kB 951 kB/s eta 0:00:01
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re
ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections
ERROR: Could not find a version that satisfies the requirement math (from versions: none)
ERROR: No matching distribution found for math
  Created wheel for sklearn: filename=sklearn-0.0.post5-py3-none-any.whl size=2360 sha256=2976b85d21c6dea719eef8fbce3f56cd1552ac8c696b23097610be78f8cfec46
  Stored in directory: /home/jovyan/.cache/pip/wheels/36/49/c9/2374f1dee1b599effabf63d948635e6608f62d0ccde027b7e2
Successfully built sklearn
     |████████████████████████████████| 226 kB 1.0 MB/s eta 0:00:01
  

In [ ]:
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split,cross_val_score,cross_val_predict
import numpy as np
import optuna
import re
from collections import Counter
from sklearn.metrics import accuracy_score,recall_score, fbeta_score,roc_auc_score,make_scorer
from sklearn.metrics import f1_score, precision_score,classification_report,balanced_accuracy_score
import nltk
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords
import torch
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from math  import ceil
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils import shuffle
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import spacy
from tqdm import tqdm
from nltk.corpus import wordnet
from sklearn.tree import DecisionTreeClassifier
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import RocCurveDisplay,ConfusionMatrixDisplay


In [ ]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
warnings.filterwarnings('ignore')


In [ ]:
# функция для подсчетов типов
def counter_of_type(data):
    return Counter(data.dtypes)


In [ ]:
# Функция для ознакомления с данными
def preview(data,n_row=4,
             head=True,
             info=True,
             columns=True,
             shape=True,
             isna=True,
             duplicated=True,
             dtypes_=True,
            describe_=True,
           clear_duples=True,
           lower_and_sep_col=True):
    global dct_nan
    global df_nan_columns
    dct_nan = {}
    if head:
        print(f'First {n_row} row(s) in the head of data')
        display(data.head(n_row))
        print('-'*30)
    if info:
        print('Data Information:')
        display(data.info())
        print('-'*30)
    if columns:
        print('Data columns:')
        display(data.columns)
        if lower_and_sep_col:
            COLUMN_REG = "(\w) (\w)"
            repl = r"\1_\2"
            data.columns = list(map(lambda x: re.sub(COLUMN_REG,repl,x).lower(),data.columns))
            print('Lowered and sepperated by words columns:')
            print(data.columns)
            print('-'*30)
    if shape:
        print('Data shape:')
        display(data.shape)
        print('-'*30)
    if isna:
        print('Data isna by row mean:')
        display(data.isna().mean())
        print()
        tmp = data.isna().mean()
        for i in range(len(tmp)):

            if tmp[i] > 0:
                dct_nan[tmp.index[i]] = tmp[i]
        if tmp.sum() > 0:
            df_nan_columns = pd.DataFrame.from_dict(dct_nan.items())
            df_nan_columns.columns = ['column','rate_of_nan']
            display(df_nan_columns)
        print('-'*30)
    if duplicated:
        print('Data clear duplicates sum:')
        duplets = data.duplicated().sum()
        display(duplets)
        if duplets >1 and clear_duples:
            data.drop_duplicates(inplace=True)
            print('Successful delete of {0} duplicates'.format(duplets))
        elif duplets == 1 and clear_duples:
            data.drop_duplicates(inplace=True)
            print('Successful delete of {0} duplicate'.format(duplets))
        print('-'*30)
    if dtypes_:
        print('Data dtypes:')
        display(data.dtypes)
        print('Number of each type:')
        print(counter_of_type(data))
        print('-'*30)
    if describe_:
        print('Major statistics for data:')
        display(data.describe())


In [ ]:
data = pd.read_csv('/datasets/toxic_comments.csv',index_col=0)

In [ ]:
preview(data,lower_and_sep_col=False)

First 4 row(s) in the head of data


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0


------------------------------
Data Information:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


None

------------------------------
Data columns:


...

0

------------------------------
Data dtypes:


text     object
toxic     int64
dtype: object

Number of each type:
Counter({dtype('O'): 1, dtype('int64'): 1})
------------------------------
Major statistics for data:


,toxic
count,159292.000000
mean,0.101612
std,0.302139
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [ ]:
data['toxic'].mean()

0.10161213369158527

In [ ]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [ ]:
LETTER_CLEAN = r"[^a-zA-Z']"


In [ ]:
sep = ' '

In [ ]:
def cleaner(text, cleaner_):
    return re.sub(cleaner_, ' ', text)

In [ ]:


data['text']= data['text'].apply(cleaner, cleaner_=LETTER_CLEAN)

In [ ]:
# Удаление стоп-слов
stop_words = set(stopwords.words("english"))

In [ ]:


def tokenise_text(text):
    # Токенизация текста
    pattern = r"\b\w+(?:'\w+)?\b"
    tokens = re.findall(pattern, text.lower().strip())
    return tokens


In [ ]:
data['text'] = data['text'].apply(tokenise_text)

In [ ]:


def remove_stopwords(text):
    filtered_tokens = [token for token in text if token not in stop_words]  # Удаление стоп-слов
    return ' '.join(filtered_tokens)


In [ ]:
data['text'] =data['text'].apply(remove_stopwords)

In [ ]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [ ]:
data['text']

0         explanation edits made username hardcore metal...
1         d'aww matches background colour i'm seemingly ...
2         hey man i'm really trying edit war guy constan...
3         can't make real suggestions improvement wonder...
4                      sir hero chance remember page that's
                                ...                        
159446    second time asking view completely contradicts...
159447                 ashamed horrible thing put talk page
159448    spitzer umm theres actual article prostitution...
159449    looks like actually put speedy first version d...
159450    really think understand came idea bad right aw...
Name: text, Length: 159292, dtype: object

In [ ]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
def lemmatize_text(text):
    lemmatizer = nltk.WordNetLemmatizer()
    lemmatized_tokens = []
    for token in nltk.word_tokenize(text):
        lemmatized_token = lemmatizer.lemmatize(token, get_wordnet_pos(token))
        lemmatized_tokens.append(lemmatized_token)
    return ' '.join(lemmatized_tokens)



In [ ]:

data['lemm_text'] = data['text'].apply(lambda x: lemmatize_text(x))


In [ ]:
# Вывод первых 5 строк с отфильтрованными данными
data['lemm_text'].head()

0    explanation edits make username hardcore metal...
1    d'aww match background colour i 'm seemingly s...
2    hey man i 'm really try edit war guy constantl...
3    ca n't make real suggestion improvement wonder...
4                sir hero chance remember page that 's
Name: lemm_text, dtype: object

## Обучение

In [ ]:
# для тестовой и валидационной выборках лучшей модели
def metrics_check(target_valid,predict_valid):
    print(f'Accuracy для тестовой выборки: {accuracy_score(target_valid,predict_valid)}')
    print('-'*100)
    print(f'Balanced accuracy для тестовой выборки: {balanced_accuracy_score(target_valid,predict_valid)}')
    print('-'*100)
    print(f'F1-score для тестовой выборки: {f1_score(target_valid,predict_valid)}')
    print('-'*100)
    print(f'Precision score для тестовой выборки: {precision_score(target_valid,predict_valid)}')
    print('-'*100)
    print(f'Recall score для тестовой выборки: {recall_score(target_valid,predict_valid)}')
    print('-'*100)
    print(f'F-beta score для тестовой выборки: {fbeta_score(target_valid,predict_valid,beta=1)}')
    print('-'*100)

In [ ]:
def valuate(model_,features_train_,features_test_,target_train_,target_test_):

    model_.fit(features_train_,target_train_)
    predicted = model_.predict(features_test_)



    print('-'*30)
    print('Метрики')
    metrics_check(target_test_,predicted)

    print('-'*30)
    print('Отчет по классификации')
    print(classification_report(target_test_,predicted))

### stratify class_weight and upsampling

In [ ]:
rs=7

In [ ]:
data

,text,toxic,lemm_text
0,explanation edits made username hardcore metal...,0,explanation edits make username hardcore metal...
1,d'aww matches background colour i'm seemingly ...,0,d'aww match background colour i 'm seemingly s...
2,hey man i'm really trying edit war guy constan...,0,hey man i 'm really try edit war guy constantl...
3,can't make real suggestions improvement wonder...,0,ca n't make real suggestion improvement wonder...
4,sir hero chance remember page that's,0,sir hero chance remember page that 's
...,...,...,...
159446,second time asking view completely contradicts...,0,second time ask view completely contradicts co...
159447,ashamed horrible thing put talk page,0,ashamed horrible thing put talk page
159448,spitzer umm theres actual article prostitution...,0,spitzer umm there actual article prostitution ...
159449,looks like actually put speedy first version d...,0,look like actually put speedy first version de...


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   text       159292 non-null  object
 1   toxic      159292 non-null  int64 
 2   lemm_text  159292 non-null  object
dtypes: int64(1), object(2)
memory usage: 4.9+ MB


In [ ]:
features = data['lemm_text']
target = data['toxic']

In [ ]:
features_train,features_test,target_train,target_test=train_test_split(features,
                                                                       target,
                                                                       test_size=0.3,
                                                                       random_state=rs)

In [ ]:
corpus_lemm_train = features_train

count_tf_idf = TfidfVectorizer(stop_words='english',analyzer='word')


tf_idf_train = count_tf_idf.fit_transform(corpus_lemm_train)


corpus_lemm_test = features_test.values


tf_idf_test = count_tf_idf.transform(corpus_lemm_test)


In [ ]:
pipeline_oversampling = imbPipeline([
    ('tfidf', TfidfVectorizer()),
    ('oversampling', RandomOverSampler(random_state=0)),
    ('logreg', LogisticRegression(random_state=42))])


parameters = {
    'tfidf__max_df': (0.25, 0.5, 0.75),
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'logreg__C': [1,2,6]
}

grid_search_tune = HalvingGridSearchCV(pipeline_oversampling, parameters, cv=3, n_jobs=-1, scoring='f1', verbose=3)
grid_search_tune.fit(features_train, target_train)

print(f"Best score: {abs(grid_search_tune.best_score_)}, Best params: {grid_search_tune.best_params_}")

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 4129
max_resources_: 111504
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 27
n_resources: 4129
Fitting 3 folds for each of 27 candidates, totalling 81 fits
[CV 1/3] END logreg__C=1, tfidf__max_df=0.25, tfidf__ngram_range=(1, 1);, score=(train=0.967, test=0.648) total time=   5.3s
[CV 2/3] END logreg__C=1, tfidf__max_df=0.25, tfidf__ngram_range=(1, 1);, score=(train=0.980, test=0.710) total time=   2.1s
[CV 3/3] END logreg__C=1, tfidf__max_df=0.25, tfidf__ngram_range=(1, 1);, score=(train=0.963, test=0.653) total time=   3.0s
[CV 1/3] END logreg__C=1, tfidf__max_df=0.25, tfidf__ngram_range=(1, 2);, score=(train=0.996, test=0.625) total time=   7.2s
[CV 2/3] END logreg__C=1, tfidf__max_df=0.25, tfidf__ngram_range=(1, 2);, score=(train=0.998, test=0.706) total time=   7.4s
[CV 3/3] END logreg__C=1, tfidf__max_df=0.25, tfidf__ngram_range=(1, 2);, score=(train=0.998, test=0.596) tota

Best score: 0.7644088944508399, Best params: {'logreg__C': 6, 'tfidf__max_df': 0.75, 'tfidf__ngram_range': (1, 1)}

In [ ]:
tfidf_max = 0.75
ngram_range = (1, 1)
corpus_lemm_train = features_train
count_tf_idf = TfidfVectorizer(stop_words='english', analyzer='word', max_df=tfidf_max, ngram_range=ngram_range)
tf_idf_train = count_tf_idf.fit_transform(corpus_lemm_train)
corpus_lemm_test = features_test.values
tf_idf_test = count_tf_idf.transform(corpus_lemm_test)


In [ ]:
clf_lr = LogisticRegression(random_state=rs, solver='sag',class_weight='balanced')
parametrs_lr = { 'C': range (1, 11,2),
              'max_iter': range (10,15,2),
                'class_weight': [None, 'balanced']
              }

grid_lr = GridSearchCV(clf_lr, parametrs_lr, cv=3, scoring='f1')
grid_lr.fit(tf_idf_train, target_train)
grid_lr.best_params_

{'C': 9, 'class_weight': None, 'max_iter': 10}

In [ ]:
grid_lr.best_score_

0.759354089351727

In [ ]:
model= LogisticRegression(random_state=rs, solver='sag',class_weight='balanced',max_iter=12,C=4)

In [ ]:
model.fit(tf_idf_train,target_train)

LogisticRegression(C=4, class_weight='balanced', max_iter=12, random_state=7,
                   solver='sag')

In [ ]:
prediction = model.predict(tf_idf_test)

In [ ]:
score_test= f1_score(prediction,target_test)

In [ ]:
score_test

0.743801652892562

In [ ]:
def lr_best_hold(model):
    # Определение пороговых значений
    thresholds = np.arange(0.3, 0.8, 0.1)

    best_score = 0
    best_threshold = 0

    # Перебор порогов и выбор лучшего на основе F1-меры

    cross_proba = cross_val_predict(model, tf_idf_train, target_train, method='predict_proba')
    for threshold in thresholds:
        # Пороговое значение для классификации
        y_pred = (cross_proba[:, 1] > threshold).astype(int)

        # Вычисление F1-меры
        score = f1_score(target_train, y_pred)

        # Проверка, является ли текущая F1-мера лучшей
        if score > best_score:
            best_score = score
            best_threshold = threshold

    # Вывод лучшего порога и соответствующей F1-меры
    print('Лучший порог:', best_threshold)
    print('Лучшая F1-мера:', best_score)

In [ ]:
lr_best_hold(model)

Лучший порог: 0.7000000000000002
Лучшая F1-мера: 0.7695666014825593


**Вывод:**

Классическая модель без каких-либо противодействий дизбалансу дает **f1 = 0.77** на тренировочной с порогом 0.7. Далее протестируем модели, в которых будем использовать stratify и весы .

In [ ]:

train_class_weights = compute_class_weight(class_weight='balanced', classes=[0, 1], y=target_train)
print("Train Class weights:", train_class_weights)
test_class_weights = compute_class_weight(class_weight='balanced', classes=[0, 1], y=target_test)
print("Test Class weights:", test_class_weights)

Train Class weights: [0.55699086 4.88666842]
Test Class weights: [0.55553231 5.00188403]


In [ ]:
train_class_weights = {0:train_class_weights[0],1:train_class_weights[1]}

In [ ]:
test_class_weights = {0:test_class_weights[0],1:test_class_weights[1]}

### DecisionTreeClassifier

In [ ]:

def objective(trial):
    # Определение гиперпараметров для оптимизации
    max_depth = trial.suggest_int('max_depth', 3, 10)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])

    # Создание и обучение модели
    model = DecisionTreeClassifier(max_depth=max_depth, min_samples_split=min_samples_split, criterion=criterion)

    # Выполнение кросс-валидации
    scorer = make_scorer(f1_score)
    scores = cross_val_score(model, tf_idf_train, target_train, cv=3, scoring='f1')

    # Вычисление среднего значения F-меры по всем фолдам
    mean_score = scores.mean()

    return mean_score





In [ ]:

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)



[I 2023-06-08 19:32:25,794] A new study created in memory with name: no-name-cade8cb0-139c-46b7-8dea-0a9bc5fb6977
[I 2023-06-08 19:32:52,910] Trial 0 finished with value: 0.5837234646552739 and parameters: {'max_depth': 8, 'min_samples_split': 2, 'criterion': 'gini'}. Best is trial 0 with value: 0.5837234646552739.
[I 2023-06-08 19:33:17,061] Trial 1 finished with value: 0.5316950613879688 and parameters: {'max_depth': 5, 'min_samples_split': 7, 'criterion': 'gini'}. Best is trial 0 with value: 0.5837234646552739.
[I 2023-06-08 19:33:47,773] Trial 2 finished with value: 0.6052682781092263 and parameters: {'max_depth': 10, 'min_samples_split': 2, 'criterion': 'gini'}. Best is trial 2 with value: 0.6052682781092263.
[I 2023-06-08 19:34:10,435] Trial 3 finished with value: 0.5096818006733916 and parameters: {'max_depth': 4, 'min_samples_split': 2, 'criterion': 'gini'}. Best is trial 2 with value: 0.6052682781092263.
[I 2023-06-08 19:34:41,716] Trial 4 finished with value: 0.59211903993105

In [ ]:
best_params = study.best_params
best_score = study.best_value

print("Лучшие гиперпараметры:", best_params)
print("Лучшее значение F-меры:", best_score)

Лучшие гиперпараметры: {'max_depth': 10, 'min_samples_split': 8, 'criterion': 'gini'}
Лучшее значение F-меры: 0.6056588213494216


### RandomForest

In [ ]:


#  с помощью библиотеки optuna
#  подберем лучшие гиперпараметры
# максимизируя F1-score
def objective(trial):
    clf_name = trial.suggest_categorical('classifier',['RandomForest'])
    if clf_name == 'RandomForest':
        max_depth = trial.suggest_int('max_depth',5,15)
        n_estimators = trial.suggest_int('n_estimators',5,150)
        criterion = trial.suggest_categorical('criterion',['gini','entropy'])
        clf_obj = RandomForestClassifier(criterion=criterion,
                                         max_depth=max_depth,
                                         n_estimators=n_estimators,
                                         class_weight=train_class_weights,
                                         random_state=rs)

    # Определяем метрику, на основе которой будет производиться оценка качества
    scorer = make_scorer(f1_score)
    clf_obj.fit(tf_idf_train,target_train)
    # Выполняем кросс-валидацию
    scores = cross_val_score(clf_obj, tf_idf_train, target_train, cv=3, scoring=scorer)
    # Вычисляем среднее значение F-меры по всем фолдам
    mean_score = scores.mean()
    return mean_score

In [ ]:
np.random.seed(7)
if __name__ == '__main__':
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials =25)
    print(study.best_trial)

[I 2023-06-08 19:41:39,783] A new study created in memory with name: no-name-2190e65e-abd0-4ad6-afaf-ff04cdec6153
[I 2023-06-08 19:42:37,113] Trial 0 finished with value: 0.3665858035984691 and parameters: {'classifier': 'RandomForest', 'max_depth': 9, 'n_estimators': 106, 'criterion': 'entropy'}. Best is trial 0 with value: 0.3665858035984691.
[I 2023-06-08 19:42:41,514] Trial 1 finished with value: 0.25667370376463533 and parameters: {'classifier': 'RandomForest', 'max_depth': 7, 'n_estimators': 10, 'criterion': 'gini'}. Best is trial 0 with value: 0.3665858035984691.
[I 2023-06-08 19:43:13,222] Trial 2 finished with value: 0.3470431860919608 and parameters: {'classifier': 'RandomForest', 'max_depth': 9, 'n_estimators': 57, 'criterion': 'entropy'}. Best is trial 0 with value: 0.3665858035984691.
[I 2023-06-08 19:44:15,939] Trial 3 finished with value: 0.36178814805394527 and parameters: {'classifier': 'RandomForest', 'max_depth': 9, 'n_estimators': 115, 'criterion': 'gini'}. Best is 

FrozenTrial(number=21, state=TrialState.COMPLETE, values=[0.4034742026940292], datetime_start=datetime.datetime(2023, 6, 8, 20, 9, 6, 541945), datetime_complete=datetime.datetime(2023, 6, 8, 20, 11, 2, 594899), params={'classifier': 'RandomForest', 'max_depth': 14, 'n_estimators': 146, 'criterion': 'gini'}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'classifier': CategoricalDistribution(choices=('RandomForest',)), 'max_depth': IntDistribution(high=15, log=False, low=5, step=1), 'n_estimators': IntDistribution(high=150, log=False, low=5, step=1), 'criterion': CategoricalDistribution(choices=('gini', 'entropy'))}, trial_id=21, value=None)


FrozenTrial(number=25, state=TrialState.COMPLETE, values=[0.8352434645068002], datetime_start=datetime.datetime(2023, 6, 7, 20, 21, 36, 592299), datetime_complete=datetime.datetime(2023, 6, 7, 20, 28, 5, 742428), params={'classifier': 'RandomForest', 'max_depth': 15, 'n_estimators': 139, 'criterion': 'gini'}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'classifier': CategoricalDistribution(choices=('RandomForest',)), 'max_depth': IntDistribution(high=15, log=False, low=5, step=1), 'n_estimators': IntDistribution(high=150, log=False, low=5, step=1), 'criterion': CategoricalDistribution(choices=('gini', 'entropy'))}, trial_id=25, value=None)

### Logistic

In [ ]:
clf_lr = LogisticRegression(random_state=rs, solver='sag',class_weight='balanced')
parametrs_lr = { 'C': range (1, 11,2),
              'max_iter': range (10,15,2),
                'class_weight': [None, 'balanced']
              }

grid_lr = GridSearchCV(clf_lr, parametrs_lr, cv=3, scoring='f1')
grid_lr.fit(tf_idf_train, target_train)
grid_lr.best_params_

{'C': 9, 'class_weight': None, 'max_iter': 10}

In [ ]:
grid_lr.best_score_

0.759354089351727

In [ ]:
# создание экземпляра логистической регрессии
best_model = LogisticRegression(C=1,penalty='l2',solver='sag',class_weight='balanced',random_state=rs)
best_model.fit(tf_idf_train,target_train)
print('F1-score на обучающей выборке:', f1_score(target_train,best_model.predict(tf_idf_train)))


F1-score на обучающей выборке: 0.8313689936536718


In [ ]:
f1_score(target_test,best_model.predict(tf_idf_test))

0.7401358047348138

In [ ]:
lr_best_hold(best_model)

Лучший порог: 0.7000000000000002
Лучшая F1-мера: 0.7704041296563271


## Выводы

In [ ]:
model

LogisticRegression(C=4, class_weight='balanced', max_iter=12, random_state=7,
                   solver='sag')

In [ ]:
model.fit(tf_idf_train,target_train)

LogisticRegression(C=4, class_weight='balanced', max_iter=12, random_state=7,
                   solver='sag')

In [ ]:
prediction = model.predict(tf_idf_test)

In [ ]:
proba = model.predict_proba(tf_idf_test)

threshold = 0.7
predictions = (proba[:, 1] >= threshold).astype(int)

In [ ]:
model

LogisticRegression(C=4, class_weight='balanced', max_iter=12, random_state=7,
                   solver='sag')

In [ ]:
f1_score(predictions,target_test)

0.7705160744500846

Лучшая модель:

LogisticRegression(C=4, class_weight='balanced', max_iter=12, random_state=7,
                   solver='sag')
                   
Классическая модель без каких-либо противодействий дизбалансу дает f1 = 0.77.





Лучшая F1-мера на тестовой выборке:

F1-score для тестовой выборки: 0.77






Модель с statify, весами и дала максимально 0.769

